In [33]:
import numpy as np
import pandas as pd
import pyarrow as pa
import sys

from pathlib import Path

import pathhelper

In [3]:
print("Python: " + sys.version)
print("Pandas: " + pd.__version__)

Python: 3.11.10 | packaged by conda-forge | (main, Sep 30 2024, 18:08:57) [GCC 13.3.0]
Pandas: 2.2.3


In [7]:
data_dir = pathhelper.data_dir(".")
filename = data_dir / "pedestrians_cyclists/Trails_Counters_Pedestrians_Cyclists.csv"

at_uses_df = pd.read_csv(filename, engine="pyarrow")
at_uses_df.columns = at_uses_dfa.columns.str.lower()

12M rows

In [14]:
at_uses_dfa.columns

Index(['', 'datetime', 'count', 'site_name', 'site_id', 'flow_id', 'flow_name',
       'user_type', 'direction', 'year', 'month', 'day', 'hour', 'minute'],
      dtype='object')

# Use Pyarrow

In [9]:
at_uses_df["count"] = at_uses_df["count"].fillna(0)

In [ ]:
# Narrowing columns
def narrow(df, column, type):
    if df[column].max() > np.iinfo(np.int16).max:
        raise OverflowError()
    return df[column].astype(type)

at_uses_df.drop("unnamed: 0", axis=1, inplace=True)

at_uses_df["count"] = narrow(at_uses_df, "count", "Int16")  # Nullable
at_uses_df["year"] = narrow(at_uses_df, "year", "int16")
at_uses_df["month"] = narrow(at_uses_df, "month", "int8")
at_uses_df["day"] = narrow(at_uses_df, "day", "int8")
at_uses_df["hour"] = narrow(at_uses_df, "hour", "int8")
at_uses_df["minute"] = narrow(at_uses_df, "minute", "int8")

at_uses_df["site_name"] = at_uses_df["site_name"].astype("category")
at_uses_df["site_id"] = at_uses_df["site_id"].astype("category")
at_uses_df["flow_id"] = at_uses_df["flow_id"].astype("category")
at_uses_df["flow_name"] = at_uses_df["flow_name"].astype("category")
at_uses_df["user_type"] = at_uses_df["user_type"].astype("category")
at_uses_df["direction"] = at_uses_df["direction"].astype("category")

In [17]:
hourly_df = at_uses_df.groupby(by=["site_name", "site_id", "flow_id", "flow_name", "user_type", "direction", "year", "month", "day", "hour"])["count"].sum()
hourly_df = hourly_df.reset_index()

In [ ]:
hourly_df["dayofweek"] = pd.to_datetime(hourly_df[['year', 'month', 'day']]).dt.dayofweek
hourly_df["week"] = pd.to_datetime(hourly_df[['year', 'month', 'day']]).dt.isocalendar().week

In [18]:
filename = data_dir / "pedestrians_cyclists/Trails_Counters_Pedestrians_Cyclists-hourly"
hourly_df.to_csv(filename.with_suffix('.csv'))
hourly_df.to_feather(filename.with_suffix('.feather'))